In [1]:
# cd("../../../temp")

In [2]:
# ]activate temp

# Lab 3: Asymptotic Properties of Best Linear Predictor and Partialling Out

To see how partialling out functions both at the population and at the sample level, we will first review the asymptotic properties of the BLP that we defined in previous classes.

First we load the libraries.

In [3]:
using GLM

Recall the usual linear model. We have a collection of random variables $(Y, X) \in \mathbb{R}\times\mathbb{R}^p$, with some joint distribution $\mathcal{D}$, and . We can build a linear approximation of $Y$ given $X$ at the population level by finding a $\beta$ such that

\begin{gather*}
\beta \in \arg\min_{b\in\mathbb{R}^p}E\left[\left(Y-Xb\right)^2\right]
\end{gather*}

However, when working with samples, we estimate $\beta$ by building $\hat{\beta}$ such that

\begin{gather*}
\hat{\beta} = \arg\min_{b\in\mathbb{R}^p}\mathbb{E}_n\left[\left(Y-Xb\right)^2\right]
\end{gather*}

Like in the previous lab, we will approximate $Y$ drawn from the following data generating process

\begin{align*}
Y &= e^{4X} + \epsilon_Y\\
X &= \epsilon_X\\
\epsilon_Y &\sim\mathcal{N}(0, 1)\\
\epsilon_X&\sim\mathcal{U}(0, 1)
\end{align*}

In [4]:
N = 10_000_000
X = rand(Float64, N)
epsilon_y = randn(N)
y = exp.(4 * X) + epsilon_y;

Remember that our initial goal is to approximate $Y$ through some linear function, which in this case means we want to approximate $g(X) = e^{4X}$. To do this, we will use a third degree polynomial like in last lab, meaning we will define $Z = (X, X^2, X^3)$ and use the least squares method to find $m(Z)\approx g(X)$. With this approximation we have that

\begin{gather*}
Y = m(Z) + \varepsilon\\
m(Z) = Z\gamma_{YZ}
\end{gather*}

where $\gamma_{YZ}$ is the vector of coefficients. This time, we will take a closer look at these coefficients.

In [5]:
Z = [X;; X .^ 2;; X .^ 3];

In [6]:
model = lm(Z, y);
print(model)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
──────────────────────────────────────────────────────────────────
       Coef.  Std. Error         t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────
x1   23.2561   0.0060268   3858.79    <1e-99    23.2443    23.268
x2  -67.5153   0.0186748  -3615.31    <1e-99   -67.5519   -67.4787
x3   97.1323   0.0138112   7032.86    <1e-99    97.1052    97.1594
──────────────────────────────────────────────────────────────────


We now pick three samples: a small one (about 50 observations), a medium one (about 250 observations) and a large (close to 10 000 observations)

In [7]:
sampling_values = rand(Float64, N)
small_sample = sampling_values .< .000005
medium_sample = sampling_values .< .000025
large_sample = sampling_values .< .001

println("Small sample size: ", sum(small_sample))
println("Medium sample size: ", sum(medium_sample))
println("Large sample size: ", sum(large_sample))

Small sample size: 51
Medium sample size: 253
Large sample size: 10117


Our $\hat{\beta}$ should more closely approximate $\beta$'s distribution as $n$ grows. We can see this when we fit a model with all the population and compare it with larger and larger samples because our estimator $\hat{\beta}$ is distributed as

\begin{gather*}
\hat{\beta}\sim\mathcal{N}(\beta, \bold{V}/n)
\end{gather*}

Notice that the variance shrinks with the sample size. In the following example results, pay attention to what happens to the confidence intervals when as the sample size grows

In [8]:
lm(Z[small_sample, :], y[small_sample])

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
───────────────────────────────────────────────────────────────
       Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────
x1   24.233      2.59068   9.35    <1e-11    19.0241    29.4419
x2  -70.7187     8.29445  -8.53    <1e-10   -87.3958   -54.0416
x3   99.5038     6.27306  15.86    <1e-19    86.891    112.117
───────────────────────────────────────────────────────────────


In [9]:
lm(Z[medium_sample, :], y[medium_sample])

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
────────────────────────────────────────────────────────────────
       Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────
x1   21.1903     1.13983   18.59    <1e-48    18.9454    23.4352
x2  -60.1353     3.53463  -17.01    <1e-42   -67.0967   -53.1738
x3   91.3997     2.62785   34.78    <1e-97    86.2241    96.5752
────────────────────────────────────────────────────────────────


In [10]:
lm(Z[large_sample, :], y[large_sample])

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
─────────────────────────────────────────────────────────────────
       Coef.  Std. Error        t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────
x1   23.4295    0.188745   124.13    <1e-99    23.0595    23.7995
x2  -68.0767    0.58616   -116.14    <1e-99   -69.2257   -66.9277
x3   97.5561    0.434518   224.52    <1e-99    96.7043    98.4078
─────────────────────────────────────────────────────────────────


Now that we have reviewed this, we can add another variable to the mix. This variable will be jointly distributed with X and with Y. We will define the data generating process as:

\begin{align*}
Y &= \theta D + e^{4X} + \epsilon_Y\\
D &= e^X - \log{(X)} + \epsilon_D\\
X &= \epsilon_X\\
\epsilon_Y, \epsilon_D&\sim\text{i.i.d.}\mathcal{N}(0, 1)\\
\epsilon_X&\sim\mathcal{U}(0, 1)
\end{align*}

In [11]:
epsilon_d = randn(N)
D = exp.(X) - log.(X) + epsilon_d
y = 3 * D + exp.(4 * X) + epsilon_y;

We will focus on understanding partialling out with this example. We will go step by step to help illustrate each step of the partialling out process and how we get the results. First, let's take a look at the coefficients for our regressors when they are jointly estimated, meaning that we find the BLP of the form

\begin{gather*}
Y = \theta D + Z\gamma + \varepsilon \quad\quad(1)
\end{gather*}

In [12]:
base_model_results = lm([D;; Z], y)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
────────────────────────────────────────────────────────────────────
        Coef.   Std. Error         t  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────
x1    3.01116  0.000216791  13889.69    <1e-99    3.01073    3.01158
x2   23.0007   0.00780718    2946.09    <1e-99   22.9854    23.016
x3  -66.948    0.0216833    -3087.54    <1e-99  -66.9905   -66.9055
x4   96.7792   0.0154198     6276.29    <1e-99   96.749     96.8094
────────────────────────────────────────────────────────────────────


Here we have a linear approximation that uses all the variables at the same time, and all coefficients are estimated jointly. As before, this means that

\begin{gather*}
Y = \hat{\theta}D + Z\hat{\gamma} + \varepsilon
\end{gather*}

We will save the residuals $\varepsilon$ of this regression for later

In [13]:
y_residuals = residuals(base_model_results);

As we saw in class, partialling out consists of computing the residuals $\~V$ for each variable $V$ in our regression with some collection of regressors $W$.

\begin{gather*}
\~V=V-\hat{V}
\end{gather*}

Where $\hat{V}=m(W)$ is a prediction rule for $V$ with $W$. The vectors $\~V$ are called the "residuals of $V$ on $W$." In the following examples, we will focus on the best linear predictor built with $Z = (X, X^2, X^3)$. This means we estimate

\begin{align*}
Y =& Z\gamma_{ZY} + \varepsilon_{ZY}\\
D =& Z\gamma_{ZD} + \varepsilon_{ZD}\\
&\text{etc...}
\end{align*}

After estimating these linear predictors, we build the residuals

\begin{align*}
\tilde{Y} =& Y - Z\hat{\gamma}_{ZY}\\
\tilde{D} =& D - Z\hat{\gamma}_{ZD}\\
&\text{etc...}
\end{align*}

We will be building these residuals for all the variables in our regression on our collection of constructed regressors $Z$, starting with the residuals of $Y$ on $Z$, $\~Y$

In [14]:
y_Z_model = lm(Z, y)
y_Z_residuals = residuals(y_Z_model)
y_Z_model

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
───────────────────────────────────────────────────────────────────
        Coef.  Std. Error         t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────
x1    91.9464   0.0271454   3387.18    <1e-99    91.8932    91.9996
x2  -220.056    0.0841134  -2616.18    <1e-99  -220.22    -219.891
x3   192.074    0.0622072   3087.66    <1e-99   191.952    192.196
───────────────────────────────────────────────────────────────────


$\~D$:

In [15]:
D_Z_model = lm(Z, D)
D_Z_residuals = residuals(D_Z_model)
D_Z_model

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
──────────────────────────────────────────────────────────────────
       Coef.  Std. Error         t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────
x1   22.8967   0.00879     2604.86    <1e-99    22.8795    22.914
x2  -50.8467   0.0272369  -1866.83    <1e-99   -50.9001   -50.7934
x3   31.6474   0.0201434   1571.10    <1e-99    31.6079    31.6869
──────────────────────────────────────────────────────────────────


$\~X$:

In [16]:
X1_Z_model = lm(Z, Z[:, 1])
X1_Z_residuals = residuals(X1_Z_model)
X1_Z_model

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
────────────────────────────────────────────────────────────────────────────────────────
           Coef.   Std. Error                    t  Pr(>|t|)     Lower 95%     Upper 95%
────────────────────────────────────────────────────────────────────────────────────────
x1   1.0          4.16345e-16  2401854690568643.00    <1e-99   1.0           1.0
x2  -4.02448e-12  1.2901e-15              -3119.52    <1e-99  -4.02701e-12  -4.02195e-12
x3   2.8532e-12   9.54109e-16              2990.43    <1e-99   2.85133e-12   2.85507e-12
────────────────────────────────────────────────────────────────────────────────────────


$\tilde{X^2}$:

In [17]:
X2_Z_model = lm(Z, Z[:, 2])
X2_Z_residuals = residuals(X2_Z_model)
X2_Z_model

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────
          Coef.   Std. Error                     t  Pr(>|t|)    Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────────────
x1  2.54624e-14  2.73513e-17                930.94    <1e-99  2.54088e-14  2.5516e-14
x2  1.0          8.47515e-17  11799200089591544.00    <1e-99  1.0          1.0
x3  1.15358e-13  6.26791e-17               1840.46    <1e-99  1.15235e-13  1.15481e-13
──────────────────────────────────────────────────────────────────────────────────────


$\tilde{X^3}$:

In [18]:
X3_Z_model = lm(Z, Z[:, 3])
X3_Z_residuals = residuals(X3_Z_model)
X3_Z_model

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
────────────────────────────────────────────────────────────────────────────────────────
           Coef.   Std. Error                    t  Pr(>|t|)     Lower 95%     Upper 95%
────────────────────────────────────────────────────────────────────────────────────────
x1  -2.10133e-13  6.95799e-17             -3020.02    <1e-99  -2.10269e-13  -2.09996e-13
x2   6.70747e-13  2.15602e-16              3111.04    <1e-99   6.70325e-13   6.7117e-13
x3   1.0          1.59452e-16  6271497774100207.00    <1e-99   1.0           1.0
────────────────────────────────────────────────────────────────────────────────────────


$\tilde{\epsilon_Y}$

In [19]:
residuals_Z_model = lm(Z, y_residuals)
residuals_Z_residuals = residuals(residuals_Z_model)
residuals_Z_model

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
────────────────────────────────────────────────────────────────────
           Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────
x1  -5.09935e-14   0.006026   -0.00    1.0000  -0.0118108  0.0118108
x2  -1.60043e-11   0.0186723  -0.00    1.0000  -0.0365971  0.0365971
x3   1.94535e-11   0.0138094   0.00    1.0000  -0.0270659  0.0270659
────────────────────────────────────────────────────────────────────


Now let us divert our attention to the relationship between the base model and a model constructed with the partialled out variables. The partialling out operation is linear, meaning that

\begin{align*}
A = bB + cC \Rightarrow \tilde{A} = b\tilde{B} + c\tilde{C}
\end{align*}

If we apply it to our initial BLP equation (1)

\begin{gather*}
\tilde{Y} = \theta \tilde{D} + \tilde{Z}\gamma + \tilde{\varepsilon} \quad\quad(2)
\end{gather*}

We can verify this by computing the right side of this equation:




In [20]:
right_side_residuals = fitted(lm([D_Z_residuals;; X1_Z_residuals;; X2_Z_residuals;; X3_Z_residuals], y_Z_residuals)) + residuals_Z_residuals

10000000-element Vector{Float64}:
  5.566551636980341
  0.1531489908619079
 -3.0819864426918717
  1.3514383917076447
  2.191923860599478
 -0.14345285523727885
  7.868526483180237
  3.5557186452380836
 -4.795963032348929
 -1.8876307845177107
 -2.2891512877757814
 -0.4596239558425952
  0.5597421953377928
  ⋮
 -1.4085899646398072
  0.04279445825598405
 -9.503432672872334
  2.0239769658942874
 -2.525998447107768
  3.670372168169491
 -0.06832119989900687
  0.9497958289824275
  4.547840410231668
 -3.825258536814802
 -2.056882317459932
  0.5542582151265787

Now we compute the left side, which is just the residuals of $Y$ on $Z$

In [21]:
y_Z_residuals

10000000-element Vector{Float64}:
  5.56655163698003
  0.15314899085990064
 -3.0819864426931005
  1.3514383917057238
  2.1919238606000917
 -0.14345285523806872
  7.868526483180015
  3.555718645240084
 -4.795963032348879
 -1.8876307845182811
 -2.2891512877774858
 -0.4596239558440107
  0.5597421953378525
  ⋮
 -1.4085899646358158
  0.04279445825432404
 -9.50343267287315
  2.02397696589434
 -2.5259984471082237
  3.670372168167475
 -0.06832119990094299
  0.9497958289822126
  4.547840410229789
 -3.8252585368159924
 -2.0568823174615574
  0.554258215125305

If we compute the difference between the left side and the right side, we get extremely small values, which are likely due to floating point errors from our system

In [22]:
y_Z_residuals - right_side_residuals

10000000-element Vector{Float64}:
 -3.1086244689504383e-13
 -2.0072554729466674e-12
 -1.2287948436551233e-12
 -1.920907877206446e-12
  6.137312880127865e-13
 -7.898681708695676e-13
 -2.220446049250313e-13
  2.000621890374532e-12
  5.062616992290714e-14
 -5.704325900524054e-13
 -1.7044143874045403e-12
 -1.4155343563970746e-12
  5.972999872483342e-14
  ⋮
  3.991473818132363e-12
 -1.660005466419534e-12
 -8.15347789284715e-13
  5.240252676230739e-14
 -4.556355293061642e-13
 -2.0161650127192843e-12
 -1.9361179326438105e-12
 -2.149391775674303e-13
 -1.879385536085465e-12
 -1.1901590823981678e-12
 -1.6253665080512292e-12
 -1.2736478538499796e-12

Why is the notable? If we take a look at (2), we should realize that
1. $\tilde{Z}=0$ as it can perfectly "predict" its own value; and
2. $\tilde{\varepsilon}=\varepsilon$ as $\varepsilon$ is independent from (orthogonal to) X, and therefor cannot be predicted with it or with Z.

This means that we can simplify (2) to

\begin{gather*}
\tilde{Y} = \theta \tilde{D} + \varepsilon\text{,}\quad\quad(3)
\end{gather*}

meaning that we can regress $\tilde{Y}$ on $\tilde{D}$ and we will find the same result as with the jointly estimated predictor. To test this, first let's take a look again at the jointly estimated predictor.

In [23]:
base_model_results

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
────────────────────────────────────────────────────────────────────
        Coef.   Std. Error         t  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────
x1    3.01116  0.000216791  13889.69    <1e-99    3.01073    3.01158
x2   23.0007   0.00780718    2946.09    <1e-99   22.9854    23.016
x3  -66.948    0.0216833    -3087.54    <1e-99  -66.9905   -66.9055
x4   96.7792   0.0154198     6276.29    <1e-99   96.749     96.8094
────────────────────────────────────────────────────────────────────


Now, according to (3), the result of regressing the residuals should be the same as above for $D$'s coefficient $\theta$

In [24]:
lm([D_Z_residuals;;], y_Z_residuals)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
──────────────────────────────────────────────────────────────────
      Coef.   Std. Error         t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────
x1  3.01116  0.000216791  13889.69    <1e-99    3.01073    3.01158
──────────────────────────────────────────────────────────────────
